# 학습 알고리즘

- 유저의 행동데이터를 받아와서 기초 알고리즘의 가중치들 수정
- 메인 알고리즘을 작성하여 유저들의 속성과 그 행동 데이터들 만으로 가중치 설정
    - 유저의 향후 행동 데이터를 계속 학습

## 행동 데이터 (유저 선택의 결과)

- schedule_2
- 유저가 일정을 변경하는 버튼을 따로 만들고, 이 버튼(일정 확정)이 푸시 되었을 때와 그 전을 비교해서 가중치에 수정을 가한다
- 유저가 자유롭게 일정을 드래그 & 드롭으로 수정할 수 있게 만들고, 수정된 것에 대한 가중치를 수집한다 (이런 경우엔 세션 종료 시 변경 여부를 가져와서 가중치 수정을 가하는 방법이겠지 ??)
- todo를 루틴과 루틴이 아닌 것으로 구분

- 유저가 많이 수행하는 todo의 카테고리를 잡고, 해당 카테고리에서 벗어나는 카테고리를 주의 카테고리로 설정

### 랜덤 포레스트 모델

#### 초기 모델 생성

In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
import joblib

# 파일 경로 설정
file_path = './refer/output/'

# 1. 초기 모델 생성용 데이터 로드
# 사용자 정보 데이터를 CSV 파일에서 불러옵니다.
user_info = pd.read_csv(f'{file_path}random_user_info.csv')

# 초기 y 값 데이터를 CSV 파일에서 불러옵니다.
schedule_1 = pd.read_csv(f'{file_path}random_schedule_1.csv')

# base_weight 데이터를 CSV 파일에서 불러옵니다.
base_weight = pd.read_csv(f'{file_path}base_weight.csv')

# 2. 초기 y 값 설정
# schedule_1의 base_weight_no와 base_weight의 no를 매칭하여 필요한 열을 가져옵니다.
y_df = schedule_1[['base_weight_no']].merge(base_weight, left_on='base_weight_no', right_on='no')[['work', 'edu', 'free_time', 'health', 'chores', 'category_else']]

# 3. 데이터 분할
# X 변수는 사용자 정보의 특정 열들을 선택합니다.
X = user_info[['age', 'gender', 'mbti', 'job']]
# 데이터를 학습용과 테스트용으로 분할합니다.
X_train, X_test, y_train, y_test = train_test_split(X, y_df, test_size=0.2, random_state=42)

# 4. 원-핫 인코딩 및 전처리 파이프라인 설정
# 범주형 변수를 원-핫 인코딩합니다.
categorical_features = ['gender', 'mbti', 'job']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# ColumnTransformer를 사용해 범주형 변수 전처리 및 나머지 변수는 그대로 유지합니다.
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough'
)

# 5. 모델 파이프라인 구성 및 하이퍼파라미터 조정
# 파라미터 그리드를 설정합니다.
param_grid = {
    'regressor__n_estimators': [100, 200, 300],  # 트리의 개수, 더 많은 트리는 더 나은 성능을 제공할 수 있지만 훈련 시간이 늘어납니다.
    'regressor__max_features': ['sqrt', 'log2', None],  # 각 노드에서 고려할 최대 특성 수, None은 모든 특성을 사용함을 의미합니다.
    'regressor__max_depth': [10, 20, 30, None],  # 트리의 최대 깊이, None은 무제한 깊이를 의미합니다.
    'regressor__min_samples_split': [2, 5, 10],  # 내부 노드를 분할하는 데 필요한 최소 샘플 수, 값이 클수록 모델이 더 일반화됩니다.
    'regressor__min_samples_leaf': [1, 2, 4]  # 리프 노드에 있어야 하는 최소 샘플 수, 값이 클수록 모델이 더 일반화됩니다.
}

# 파이프라인을 사용하여 전처리 및 모델을 설정합니다.
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))
])

# GridSearchCV를 사용하여 하이퍼파라미터 튜닝을 수행합니다.
grid_search = GridSearchCV(
    model,
    param_grid,
    cv=3,  # 교차 검증 폴드 수
    n_jobs=-1,  # 사용할 CPU 코어 수, -1은 모든 코어 사용
    scoring='neg_mean_squared_error'  # 평가 기준, 여기서는 음의 평균 제곱 오차
)

# 모델을 학습합니다.
grid_search.fit(X_train, y_train)

# 최적의 모델을 저장합니다.
best_model = grid_search.best_estimator_
joblib.dump(best_model, f'{file_path}initial_model.pkl')

print("Initial model created and saved.")


Initial model created and saved.


#### 후기 모델

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
import joblib
from joblib import parallel_backend

# 디버깅 메시지 출력 함수
def debug_message(message):
    print(f"[디버깅] {message}")

# 파일 경로 설정
file_path = './refer/output/'

# 1. 초기 모델 생성용 데이터 로드
debug_message("사용자 정보 데이터를 불러오는 중...")
user_info = pd.read_csv(f'{file_path}random_user_info.csv')
debug_message("사용자 정보 데이터 로드 완료")

debug_message("schedule_2 데이터를 불러오는 중...")
schedule_2 = pd.read_csv(f'{file_path}random_schedule_2.csv')
debug_message("schedule_2 데이터 로드 완료")

debug_message("user_weight 데이터를 불러오는 중...")
user_weight = pd.read_csv(f'{file_path}random_user_weight.csv')
debug_message("user_weight 데이터 로드 완료")

# 2. y 값 설정
debug_message("y 값 설정 중...")
y_df = schedule_2[['user_weight_no']].merge(user_weight, on='user_weight_no')[['work', 'edu', 'free_time', 'health', 'chores', 'category_else']]
debug_message("y 값 설정 완료")

# 3. 데이터 분할
debug_message("데이터 분할 중...")
X = user_info[['age', 'gender', 'mbti', 'job']]
X_train, X_test, y_train, y_test = train_test_split(X, y_df, test_size=0.2, random_state=42)
debug_message("데이터 분할 완료")

# 4. 원-핫 인코딩 및 전처리 파이프라인 설정
debug_message("원-핫 인코딩 및 전처리 파이프라인 설정 중...")
categorical_features = ['gender', 'mbti', 'job']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)  # 'cat'은 범주형 변수 원-핫 인코딩을 수행하는 변환기
    ],
    remainder='passthrough'
)
debug_message("원-핫 인코딩 및 전처리 파이프라인 설정 완료")

# 5. 모델 파이프라인 구성 및 하이퍼파라미터 조정
debug_message("하이퍼파라미터 그리드 설정 중...")
param_grid = {
    'regressor__n_estimators': [100, 200, 300],  # 트리의 개수, 더 많은 트리는 더 나은 성능을 제공할 수 있지만 훈련 시간이 늘어남
    'regressor__max_features': ['sqrt', 'log2', None],  # 각 노드에서 고려할 최대 특성 수, None은 모든 특성을 사용함을 의미
    'regressor__max_depth': [10, 20, 30, None],  # 트리의 최대 깊이, None은 무제한 깊이를 의미
    'regressor__min_samples_split': [2, 5, 10],  # 내부 노드를 분할하는 데 필요한 최소 샘플 수, 값이 클수록 모델이 더 일반화됨
    'regressor__min_samples_leaf': [1, 2, 4]  # 리프 노드에 있어야 하는 최소 샘플 수, 값이 클수록 모델이 더 일반화됨
}
debug_message("하이퍼파라미터 그리드 설정 완료")

# 파이프라인을 사용하여 전처리 및 모델을 설정합니다.
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))
])

# GridSearchCV를 사용하여 하이퍼파라미터 튜닝을 수행합니다.
with parallel_backend('threading'):
    grid_search = GridSearchCV(
        model,
        param_grid,
        cv=3,  # 교차 검증 폴드 수
        n_jobs=-1,  # 사용할 CPU 코어 수, -1은 모든 코어 사용
        scoring='neg_mean_squared_error'  # 평가 기준, 여기서는 음의 평균 제곱 오차
    )

    # 모델을 학습합니다.
    debug_message("모델 학습 중...")
    grid_search.fit(X_train, y_train)
    debug_message("모델 학습 완료")

# 최적의 모델을 저장합니다.
best_model = grid_search.best_estimator_
joblib.dump(best_model, f'{file_path}initial_model.pkl')

debug_message("최적의 모델 저장 완료")
print("Initial model created and saved.")


#### 모델 업데이트

In [6]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
import joblib

# 파일 경로 설정
file_path = './refer/output/'

# 1. 기존 모델 로드
# 초기 모델을 파일에서 로드합니다.
model = joblib.load(f'{file_path}initial_model.pkl')

# 2. schedule_2에서 업데이트 데이터 로드
# schedule_2에서 최신 데이터를 불러옵니다.
schedule_2 = pd.read_csv(f'{file_path}random_schedule_2.csv')

# 3. user_info에서 사용자 정보 데이터 로드
# 사용자 정보 데이터를 CSV 파일에서 불러옵니다.
user_info = pd.read_csv(f'{file_path}random_user_info.csv')

# 4. 업데이트할 X, y 설정
# X 업데이트 데이터를 설정합니다.
X_update = user_info[['age', 'gender', 'mbti', 'job']]
# y 업데이트 데이터를 설정합니다.
y_update = schedule_2.iloc[-1].values.reshape(1, -1)

# 5. 원-핫 인코딩 및 전처리 파이프라인 설정
# 범주형 변수를 원-핫 인코딩합니다.
categorical_features = ['gender', 'mbti', 'job']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# ColumnTransformer를 사용해 범주형 변수 전처리 및 나머지 변수는 그대로 유지합니다.
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough'
)

# 6. 파이프라인을 사용하여 전처리 및 모델 설정
# 모델의 전처리 부분을 업데이트합니다.
model.steps[0] = ('preprocessor', preprocessor)

# 7. 모델 업데이트
# X 업데이트 데이터를 전처리합니다.
X_train_update = preprocessor.fit_transform(X_update)
# 모델을 업데이트된 데이터로 재학습시킵니다.
model.named_steps['regressor'].fit(X_train_update, y_update)

# 8. 업데이트된 모델 저장
# 업데이트된 모델을 저장합니다.
joblib.dump(model, f'{file_path}updated_model.pkl')

print("Model updated and saved.")


FileNotFoundError: [Errno 2] No such file or directory: './refer/output/schedule_2.csv'

#### 후기 업데이트 모델

In [ ]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
import joblib

# 디버깅 메시지 출력 함수
def debug_message(message):
    print(f"[디버깅] {message}")

# 파일 경로 설정
file_path = './refer/output/'

# 1. 기존 모델 및 데이터 로드
debug_message("기존 모델과 데이터를 로드하는 중...")
model = joblib.load(f'{file_path}initial_model.pkl')
user_info = pd.read_csv(f'{file_path}random_user_info.csv')
schedule_2 = pd.read_csv(f'{file_path}random_schedule_2.csv')
user_weight = pd.read_csv(f'{file_path}random_user_weight.csv')
debug_message("기존 모델과 데이터 로드 완료")

# 데이터프레임의 열 이름 출력
debug_message(f"user_weight 열 이름: {user_weight.columns}")
debug_message(f"schedule_2 열 이름: {schedule_2.columns}")

# 2. y 값 설정
debug_message("y 값 설정 중...")
y_df = schedule_2[['user_weight_no']].merge(user_weight, on='user_weight_no')[['work', 'edu', 'free_time', 'health', 'chores', 'category_else']]
debug_message("y 값 설정 완료")

# 3. 데이터 결합
debug_message("데이터 결합 중...")
X_combined = user_info.copy()
y_combined = y_df.copy()
debug_message("데이터 결합 완료")

# 4. 원-핫 인코딩 및 전처리 파이프라인 설정
debug_message("원-핫 인코딩 및 전처리 파이프라인 설정 중...")
categorical_features = ['gender', 'mbti', 'job']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)  # 'cat'은 범주형 변수 원-핫 인코딩을 수행하는 변환기
    ],
    remainder='passthrough'
)
debug_message("원-핫 인코딩 및 전처리 파이프라인 설정 완료")

# 5. 모델 파이프라인 설정 및 재학습
debug_message("모델 재학습 중...")
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(random_state=42))
])

# 모델을 전체 데이터를 사용하여 재학습합니다.
model.fit(X_combined, y_combined)
debug_message("모델 재학습 완료")

# 6. 업데이트된 모델 저장
joblib.dump(model, f'{file_path}updated_model.pkl')

debug_message("업데이트된 모델 저장 완료")
print("Random Forest model updated and saved.")


------------

##### Not for Use

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

In [ ]:
# 가독성을 위해 지수 표현(e) 변환
pd.options.display.float_format = '{:.5f}'.format

In [ ]:
file_path = './refer/output/'

In [ ]:
# 데이터 로딩
data = pd.read_csv(f'{file_path}survey_data.csv')

# 고객 선택 데이터
user_schedule = pd.read_csv(f'{file_path}user_schedule.csv')

In [ ]:
# 범주형 데이터를 원-핫 인코딩
categorical_features = ['gender', 'mbti', 'job']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# 전처리 파이프라인 설정
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough'
)

# 타겟 변수와 피처 분리
X = data[['age', 'gender', 'mbti', 'job']]
y = data[['work', 'edu', 'free_time', 'health', 'chores']]

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 모델 파이프라인 구성
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

# 모델 학습
model.fit(X_train, y_train)

# 예측 및 평가
y_pred = model.predict(X_test)

# 예측 결과를 데이터프레임으로 변환
y_pred_df = pd.DataFrame(y_pred, columns=y_test.columns)

# MSE 계산 결과를 데이터프레임으로 변환
mse = mean_squared_error(y_test, y_pred)
mse_df = pd.DataFrame([mse], columns=['MSE'])

# 결과 출력
print(mse_df)
print(y_pred_df)

### 그라디언트 부스팅 회귀 모델

#### LightGBM 빌드

#### 초기 모델 빌드

In [10]:
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from lightgbm import LGBMRegressor
import joblib
from joblib import parallel_backend

# 디버깅 메시지 출력 함수
def debug_message(message):
    print(f"[디버깅] {message}")

# 파일 경로 설정
file_path = './refer/output/'

# 1. 초기 모델 생성용 데이터 로드
debug_message("사용자 정보 데이터를 불러오는 중...")
user_info = pd.read_csv(f'{file_path}random_user_info.csv')
debug_message("사용자 정보 데이터 로드 완료")

debug_message("초기 y 값 데이터를 불러오는 중...")
schedule_1 = pd.read_csv(f'{file_path}random_schedule_1.csv')
debug_message("초기 y 값 데이터 로드 완료")

debug_message("base_weight 데이터를 불러오는 중...")
base_weight = pd.read_csv(f'{file_path}base_weight.csv')
debug_message("base_weight 데이터 로드 완료")

debug_message("schedule_2 데이터를 불러오는 중...")
schedule_2 = pd.read_csv(f'{file_path}random_schedule_2.csv')
debug_message("schedule_2 데이터 로드 완료")

debug_message("user_weight 데이터를 불러오는 중...")
user_weight = pd.read_csv(f'{file_path}random_user_weight.csv')
debug_message("user_weight 데이터 로드 완료")

# 데이터프레임의 열 이름 출력
debug_message(f"user_weight 열 이름: {user_weight.columns}")
debug_message(f"schedule_2 열 이름: {schedule_2.columns}")

# 2. y 값 설정
debug_message("y 값 설정 중...")
y_df = schedule_2[['user_weight_no']].merge(
    user_weight, on='user_weight_no'
)[['work', 'edu', 'free_time', 'health', 'chores']]
debug_message("y 값 설정 완료")

# 3. 데이터 분할
debug_message("데이터 분할 중...")
X = user_info[['age', 'gender', 'mbti', 'job']]
X_train, X_test, y_train, y_test = train_test_split(X, y_df, test_size=0.2, random_state=42)
debug_message("데이터 분할 완료")

# 4. 원-핫 인코딩 및 전처리 파이프라인 설정
debug_message("원-핫 인코딩 및 전처리 파이프라인 설정 중...")
categorical_features = ['gender', 'mbti', 'job']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)  # 'cat'은 범주형 변수 원-핫 인코딩을 수행하는 변환기
    ],
    remainder='passthrough'
)
debug_message("원-핫 인코딩 및 전처리 파이프라인 설정 완료")

# 5. 각 타겟에 대한 모델 훈련 및 저장
debug_message("하이퍼파라미터 그리드 설정 중...")

param_grid = {
    'regressor__n_estimators': [100, 200, 300],  # 부스팅 반복 횟수, 더 많은 반복은 더 나은 성능을 제공할 수 있지만, 훈련 시간 증가
    'regressor__learning_rate': [0.01, 0.05, 0.1],  # 학습률, 모델이 업데이트되는 속도 조절, 작은 값은 더 천천히 학습하지만 더 나은 일반화 제공
    'regressor__num_leaves': [31, 50, 100],  # 하나의 트리가 가질 수 있는 최대 잎사귀 수, 값이 클수록 모델의 복잡도 증가
    'regressor__max_depth': [-1, 10, 20, 30],  # 트리의 최대 깊이, -1은 무제한 깊이를 의미
    'regressor__min_child_samples': [20, 50, 100],  # 리프 노드가 되기 위한 최소 데이터 수, 값이 클수록 모델이 더 일반화됨
    'regressor__feature_fraction': [0.6, 0.8, 1.0],  # 각 트리마다 사용되는 피처 비율, 0.8은 각 트리가 무작위로 선택된 80%의 피처를 사용함을 의미
    'regressor__bagging_fraction': [0.6, 0.8, 1.0],  # 데이터 샘플링 비율, 0.8은 각 반복에서 무작위로 선택된 80%의 데이터를 사용함을 의미
    'regressor__bagging_freq': [0, 5, 10],  # 배깅 빈도, k번의 반복 후 데이터 샘플링을 수행, 0은 사용하지 않음을 의미
    'regressor__lambda_l1': [0, 0.1, 1.0],  # L1 정규화, 모델의 가중치를 0으로 만들도록 함, 과적합 방지
    'regressor__lambda_l2': [0, 0.1, 1.0]  # L2 정규화, 모델의 가중치를 작게 유지하도록 함, 과적합 방지
}


debug_message("하이퍼파라미터 그리드 설정 완료")

models = {}
# parallel_backend를 사용하여 threading 백엔드로 설정
# 멀티프로세싱 대신 멀티스레딩을 사용하여 피클링 문제 회피
with parallel_backend('threading'):
    for target in y_df.columns:
        debug_message(f"{target}에 대한 모델 훈련 중...")
        pipeline = Pipeline(steps=[
            ('preprocessor', preprocessor),
            ('regressor', LGBMRegressor(random_state=42))
        ])
        grid_search = GridSearchCV(
            pipeline,
            param_grid,
            cv=3,  # 교차 검증 폴드 수
            n_jobs=-1,  # 사용할 CPU 코어 수, -1은 모든 코어 사용
            scoring='neg_mean_squared_error'  # 평가 기준, 음의 평균 제곱 오차 사용
        )
        grid_search.fit(X_train, y_train[target])
        models[target] = grid_search.best_estimator_
        debug_message(f"{target}에 대한 모델 훈련 완료")

# 6. 모델 저장 함수
def save_model_with_versioning(models, file_path, base_filename):
    version = 1
    filename = f"{base_filename}_v{version}.pkl"
    while os.path.exists(f"{file_path}{filename}"):
        version += 1
        filename = f"{base_filename}_v{version}.pkl"
    joblib.dump(models, f"{file_path}{filename}")
    return filename

# 모델 저장
debug_message("모델 저장 중...")
saved_filename = save_model_with_versioning(models, file_path, "initial_lightgbm_models")
debug_message(f"모델 저장 완료: {saved_filename}")

print(f"Initial LightGBM models created and saved as {saved_filename}.")


[디버깅] 사용자 정보 데이터를 불러오는 중...
[디버깅] 사용자 정보 데이터 로드 완료
[디버깅] 초기 y 값 데이터를 불러오는 중...
[디버깅] 초기 y 값 데이터 로드 완료
[디버깅] base_weight 데이터를 불러오는 중...
[디버깅] base_weight 데이터 로드 완료
[디버깅] schedule_2 데이터를 불러오는 중...
[디버깅] schedule_2 데이터 로드 완료
[디버깅] user_weight 데이터를 불러오는 중...
[디버깅] user_weight 데이터 로드 완료
[디버깅] user_weight 열 이름: Index(['user_weight_no', 'work', 'edu', 'free_time', 'health', 'chores',
       'category_else', 'history_no'],
      dtype='object')
[디버깅] schedule_2 열 이름: Index(['user_id', 'user_goal', 'goal_complexity', 'goal_start_date',
       'goal_end_date', 'record_time', 'user_weight_no'],
      dtype='object')
[디버깅] y 값 설정 중...
[디버깅] y 값 설정 완료
[디버깅] 데이터 분할 중...
[디버깅] 데이터 분할 완료
[디버깅] 원-핫 인코딩 및 전처리 파이프라인 설정 중...
[디버깅] 원-핫 인코딩 및 전처리 파이프라인 설정 완료
[디버깅] 하이퍼파라미터 그리드 설정 중...
[디버깅] 하이퍼파라미터 그리드 설정 완료
[디버깅] work에 대한 모델 훈련 중...
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] lambda_l1 is set=0.1, reg_alpha=

##### 피클링 오류 발생했던 기존 코드

In [ ]:
# 5. 각 타겟에 대한 모델 훈련 및 저장
# 하이퍼파라미터 그리드를 설정합니다.
param_grid = {
    'n_estimators': [100, 200, 300],  # 부스팅 반복 횟수, 더 많은 반복은 더 나은 성능을 제공할 수 있지만, 훈련 시간이 늘어납니다.
    'learning_rate': [0.01, 0.05, 0.1],  # 학습률, 모델이 업데이트되는 속도를 조절합니다. 작은 값은 더 천천히 학습하지만 더 나은 일반화를 제공합니다.
    'num_leaves': [31, 50, 100],  # 하나의 트리가 가질 수 있는 최대 잎사귀 수, 값이 클수록 모델의 복잡도가 높아집니다.
    'max_depth': [-1, 10, 20, 30],  # 트리의 최대 깊이, -1은 무제한 깊이를 의미합니다.
    'min_child_samples': [20, 50, 100],  # 리프 노드가 되기 위한 최소 데이터 수, 값이 클수록 모델이 더 일반화됩니다.
    'feature_fraction': [0.6, 0.8, 1.0],  # 각 트리마다 사용되는 피처 비율, 0.8은 각 트리가 무작위로 선택된 80%의 피처를 사용함을 의미합니다.
    'bagging_fraction': [0.6, 0.8, 1.0],  # 데이터 샘플링 비율, 0.8은 각 반복에서 무작위로 선택된 80%의 데이터를 사용함을 의미합니다.
    'bagging_freq': [0, 5, 10],  # 배깅 빈도, k번의 반복 후 데이터 샘플링을 수행합니다. 0은 사용하지 않음을 의미합니다.
    'lambda_l1': [0, 0.1, 1.0],  # L1 정규화, 모델의 가중치를 0으로 만들도록 합니다. 과적합을 방지합니다.
    'lambda_l2': [0, 0.1, 1.0]  # L2 정규화, 모델의 가중치를 작게 유지하도록 합니다. 과적합을 방지합니다.
}

# 각 타겟에 대해 모델을 훈련하고 최적의 하이퍼파라미터를 찾습니다.
models = {}
for target in y_df.columns:
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', LGBMRegressor(random_state=42))
    ])
    grid_search = GridSearchCV(
        pipeline,
        param_grid,
        cv=3,  # 교차 검증 폴드 수
        n_jobs=-1,  # 사용할 CPU 코어 수, -1은 모든 코어 사용
        scoring='neg_mean_squared_error'  # 평가 기준, 여기서는 음의 평균 제곱 오차
    )
    grid_search.fit(X_train, y_train[target])
    models[target] = grid_search.best_estimator_

# 6. 모델 저장
joblib.dump(models, f'{file_path}initial_lightgbm_models.pkl') # joblib의 기본 설정인 Loky 사용으로 인해 피클링 오류 발생

#### 모델 업데이트

In [ ]:
import numpy as np
import pandas as pd
import os
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from lightgbm import LGBMRegressor
import joblib

# 디버깅 메시지 출력 함수
def debug_message(message):
    print(f"[디버깅] {message}")

# 파일 경로 설정
file_path = './refer/output/'

# 기존 모델 및 데이터 로드
debug_message("기존 모델과 데이터를 로드하는 중...")
models = joblib.load(f'{file_path}initial_lightgbm_models.pkl')
user_info = pd.read_csv(f'{file_path}random_user_info.csv')
schedule_1 = pd.read_csv(f'{file_path}random_schedule_1.csv')
base_weight = pd.read_csv(f'{file_path}base_weight.csv')
schedule_2 = pd.read_csv(f'{file_path}random_schedule_2.csv')
user_weight = pd.read_csv(f'{file_path}random_user_weight.csv')
debug_message("기존 모델과 데이터 로드 완료")

# 초기 y 값 설정 및 결합
debug_message("초기 y 값 설정 및 결합 중...")
y_df_1 = schedule_1[['base_weight_no']].merge(base_weight, left_on='base_weight_no', right_on='no')[['work', 'edu', 'free_time', 'health', 'chores']]
y_df_2 = schedule_2[['user_weight_no']].merge(user_weight, on='user_weight_no')[['work', 'edu', 'free_time', 'health', 'chores']]
debug_message("초기 y 값 설정 및 결합 완료")

# X 및 y 결합
debug_message("X 및 y 결합 중...")
X_combined = pd.concat([user_info, user_info])
y_combined = pd.concat([y_df_1, y_df_2])
debug_message("X 및 y 결합 완료")

# 원-핫 인코딩 및 전처리 파이프라인 설정
debug_message("원-핫 인코딩 및 전처리 파이프라인 설정 중...")
categorical_features = ['gender', 'mbti', 'job']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)  # 'cat'은 범주형 변수 원-핫 인코딩을 수행하는 변환기
    ],
    remainder='passthrough'
)
debug_message("원-핫 인코딩 및 전처리 파이프라인 설정 완료")

# 파이프라인 설정 및 모델 재학습
debug_message("모델 재학습 중...")
for target in y_combined.columns:
    debug_message(f"{target}에 대한 모델 훈련 중...")
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', LGBMRegressor(random_state=42))
    ])
    pipeline.fit(X_combined, y_combined[target])
    models[target] = pipeline
    debug_message(f"{target}에 대한 모델 훈련 완료")

# 모델 저장 함수
def save_model_with_versioning(models, file_path, base_filename):
    version = 1
    filename = f"{base_filename}_v{version}.pkl"
    while os.path.exists(f"{file_path}{filename}"):
        version += 1
        filename = f"{base_filename}_v{version}.pkl"
    joblib.dump(models, f"{file_path}{filename}")
    return filename

# 업데이트된 모델 저장
debug_message("업데이트된 모델 저장 중...")
saved_filename = save_model_with_versioning(models, file_path, "updated_lightgbm_models")
debug_message(f"업데이트된 모델 저장 완료: {saved_filename}")

print(f"LightGBM models updated and saved as {saved_filename}.")


-------------

##### Not for Use

In [ ]:
# 라이브러리 로드
import pandas as pd
from sklearn.model_selection import train_test_split
from lightgbm import LGBMRegressor
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

In [ ]:
# 데이터 로딩
data = pd.read_csv(f'{file_path}survey_data.csv')

In [ ]:
# 범주형 데이터를 원-핫 인코딩
categorical_features = ['gender', 'mbti', 'job']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# 전처리 파이프라인 설정
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough'
)

# 타겟 변수와 피처 분리
X = data[['age', 'gender', 'mbti', 'job']]
y = data[['work', 'edu', 'free_time', 'health', 'chores']]

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 전처리
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

# 각 타겟에 대한 모델 훈련 및 예측
mse = []
predictions = []
for target in y.columns:
    model = LGBMRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train[target])
    y_pred = model.predict(X_test)
    mse.append(mean_squared_error(y_test[target], y_pred))
    predictions.append(y_pred)

# 예측 결과를 데이터프레임으로 변환
y_pred_df = pd.DataFrame(zip(*predictions), columns=y.columns)

# MSE 결과 출력
for idx, target in enumerate(y.columns):
    print(f'MSE for {target}: {format(mse[idx], ".4f")}')

# 전체 MSE 계산 및 출력
overall_mse = mean_squared_error(y_test, y_pred_df)
print("Overall MSE:", format(overall_mse, ".4f"))

# 예측 결과 데이터프레임 출력, 지수 표현식 없이
print(y_pred_df.to_string(index=False, header=True, float_format="{:0.4f}".format))

#### XGBoost 빌드 (너무 메모리 많이 먹고 오래 걸려서 사용 안함)

#### 초기 모델 빌드

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import joblib

# 파일 경로 설정
file_path = 'your/file/path/'

# 1. 초기 모델 생성용 데이터 로드
# 사용자 정보 데이터를 CSV 파일에서 불러옵니다.
user_info = pd.read_csv(f'{file_path}user_info.csv')

# 초기 y 값 데이터를 CSV 파일에서 불러옵니다.
schedule_1 = pd.read_csv(f'{file_path}schedule_1.csv')

# 2. 초기 y 값 설정
# 초기 y 값으로 schedule_1 데이터를 사용합니다.
y_df = schedule_1.copy()

# 3. 데이터 분할
# X 변수는 사용자 정보의 특정 열들을 선택합니다.
X = user_info[['age', 'gender', 'mbti', 'job']]
# 데이터를 학습용과 테스트용으로 분할합니다.
X_train, X_test, y_train, y_test = train_test_split(X, y_df, test_size=0.2, random_state=42)

# 4. 원-핫 인코딩 및 전처리 파이프라인 설정
# 범주형 변수를 원-핫 인코딩합니다.
categorical_features = ['gender', 'mbti', 'job']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# ColumnTransformer를 사용해 범주형 변수 전처리 및 나머지 변수는 그대로 유지합니다.
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough'
)

# 5. 전처리
# 학습 데이터에 대해 전처리를 수행합니다.
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

# 6. 각 타겟에 대한 모델 훈련 및 저장
# 하이퍼파라미터 그리드를 설정합니다.
param_grid = {
    'n_estimators': [100, 200, 300],  # 부스팅 반복 횟수
    'learning_rate': [0.01, 0.05, 0.1],  # 학습률
    'max_depth': [3, 5, 7],  # 트리의 최대 깊이
    'min_child_weight': [1, 3, 5],  # 자식 노드를 가지기 위한 최소 가중치 합
    'subsample': [0.6, 0.8, 1.0],  # 각 트리마다 사용되는 데이터 샘플 비율
    'colsample_bytree': [0.6, 0.8, 1.0]  # 각 트리마다 사용되는 피처 샘플 비율
}

# 각 타겟에 대해 모델을 훈련하고 최적의 하이퍼파라미터를 찾습니다.
models = {}
for target in y_df.columns:
    model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
    grid_search = GridSearchCV(model, param_grid, cv=3, n_jobs=-1, scoring='neg_mean_squared_error')
    grid_search.fit(X_train, y_train[target])
    models[target] = grid_search.best_estimator_

# 7. 모델 저장
joblib.dump(models, f'{file_path}initial_xgboost_models.pkl')
joblib.dump(preprocessor, f'{file_path}preprocessor.pkl')

print("Initial XGBoost models created and saved.")

#### 모델 업데이트

In [ ]:
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import joblib

# 파일 경로 설정
file_path = 'your/file/path/'

# 1. 기존 모델 로드
# 초기 모델과 전처리기를 파일에서 로드합니다.
models = joblib.load(f'{file_path}initial_xgboost_models.pkl')
preprocessor = joblib.load(f'{file_path}preprocessor.pkl')

# 2. schedule_2에서 업데이트 데이터 로드
# schedule_2에서 최신 데이터를 불러옵니다.
schedule_2 = pd.read_csv(f'{file_path}schedule_2.csv')

# 3. user_info에서 사용자 정보 데이터 로드
# 사용자 정보 데이터를 CSV 파일에서 불러옵니다.
user_info = pd.read_csv(f'{file_path}user_info.csv')

# 4. 업데이트할 X, y 설정
# X 업데이트 데이터를 설정합니다.
X_update = user_info[['age', 'gender', 'mbti', 'job']]
# y 업데이트 데이터를 설정합니다.
y_update = schedule_2.iloc[-1].values.reshape(1, -1)

# 5. 전처리
# 업데이트 데이터에 대해 전처리를 수행합니다.
X_update = preprocessor.transform(X_update)

# 6. 모델 업데이트
for idx, target in enumerate(schedule_2.columns):
    models[target].fit(X_update, y_update[:, idx])

# 7. 업데이트된 모델 저장
joblib.dump(models, f'{file_path}updated_xgboost_models.pkl')

print("XGBoost models updated and saved.")

##### Not for Use

In [ ]:
# 라이브러리 로드
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_error

In [ ]:
# 데이터 로딩
data = pd.read_csv(f'{file_path}survey_data.csv')

In [ ]:
# 범주형 데이터를 원-핫 인코딩
categorical_features = ['gender', 'mbti', 'job']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# 전처리 파이프라인 설정
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough'
)

# 타겟 변수와 피처 분리
X = data[['age', 'gender', 'mbti', 'job']]
y = data[['work', 'edu', 'free_time', 'health', 'chores']]

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 전처리 및 변환
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

# 각 타겟에 대한 모델 훈련 및 예측 결과 저장
predictions = []
for i, target in enumerate(y.columns):
    model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)
    model.fit(X_train, y_train.iloc[:, i])
    y_pred = model.predict(X_test)
    predictions.append(y_pred)
    print(f'MSE for {target}: {mean_squared_error(y_test.iloc[:, i], y_pred)}')

# 예측 결과를 데이터프레임으로 변환
y_pred_df = pd.DataFrame(predictions).T  # Transpose to align with y_test's shape
y_pred_df.columns = y_test.columns

# 예측 데이터프레임과 실제 데이터프레임의 MSE 계산
mse = mean_squared_error(y_test, y_pred_df)
print("Overall MSE:", format(mse, ".4f"))

# 예측 결과 데이터프레임 출력, 지수 표현식 없이
print(y_pred_df.to_string(index=False, header=True, float_format="{:0.4f}".format))

### 딥러닝 모델 (시간 없어서 사용 안함 (데이터도 없어))

#### TensorFlow 빌드

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
# 데이터 로딩
data = pd.read_csv(f'{file_path}survey_data.csv')

In [ ]:
# 범주형 데이터를 원-핫 인코딩
categorical_features = ['gender', 'mbti', 'job']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# 전처리 파이프라인 설정
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)
    ],
    remainder='passthrough'
)

# 타겟 변수와 피처 분리
X = data[['age', 'gender', 'mbti', 'job']]
y = data[['work', 'edu', 'free_time', 'health', 'chores']]

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 전처리 및 변환
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [ ]:
# 딥러닝 모델 구성
model = Sequential([
    Dense(128, activation='relu', input_dim=X_train.shape[1]),
    Dense(64, activation='relu'),
    Dense(5)  # 출력 레이어: 5개 타겟 변수
])

# 모델 컴파일
model.compile(optimizer='adam', loss='mse')

# 모델 학습
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)

In [ ]:
# 예측 및 평가
y_pred = model.predict(X_test)

# MSE 계산
mse = tf.keras.losses.MeanSquaredError()
mse_value = mse(y_test, y_pred).numpy()
print("MSE:", format(mse_value, ".4f"))


----------------

## 시간 슬라이싱

- 딥러닝 모델로 목표에 대한 업무를 중요도와 긴급도를 기준으로 시간 슬라이싱
- 중요도(weight), 긴급도(fire_data, complexity)를 통해 가용 시간 하루 시간 단위로 슬라이싱
- 슬라이싱 된 작은 목표 달성시 계속, 미 달성시(부분 미달성, 전체 미달성) 슬라이싱에 다시 반영

### 딥러닝 모델

#### TensorFlow 빌드

##### 작업 중요도 예측 모델

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import mean_squared_error
import os

# 디버깅 메시지 출력 함수
def debug_message(message):
    print(f"[디버깅] {message}")

# 모델 버져닝 함수
def save_model_with_versioning(model, file_path, base_filename):
    version = 1
    filename = f"{base_filename}_v{version}.h5"
    while os.path.exists(os.path.join(file_path, filename)):
        version += 1
        filename = f"{base_filename}_v{version}.h5"
    model.save(os.path.join(file_path, filename))
    return filename

# 데이터 로드 및 전처리
file_path = './refer/output/'
debug_message("작업 중요도 데이터 로드 중...")
data = pd.read_csv(f'{file_path}data.csv')
debug_message("작업 중요도 데이터 로드 완료")

# 범주형 데이터에 대한 원-핫 인코딩
debug_message("범주형 데이터 원-핫 인코딩 중...")
encoder = OneHotEncoder()
encoded_data = encoder.fit_transform(data[['gender', 'job', 'mbti']]).toarray()
debug_message("범주형 데이터 원-핫 인코딩 완료")

# 인코딩된 데이터와 수치형 데이터를 결합
numeric_data = data[['age', 'work', 'edu', 'free_time', 'health', 'chores']].values
X = np.hstack((encoded_data, numeric_data))

# 데이터 정규화
debug_message("데이터 정규화 중...")
scaler = StandardScaler()
X = scaler.fit_transform(X)
debug_message("데이터 정규화 완료")

# 목표 변수 정의 (중요도)
y = data[['work', 'edu', 'free_time', 'health', 'chores']].values

# 데이터를 학습용과 테스트용으로 분할
debug_message("데이터 학습용 및 테스트용 분할 중...")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
debug_message("데이터 학습용 및 테스트용 분할 완료")

# 하이퍼파라미터 설정
learning_rate = 0.001  # 학습률 결정: 학습 과정에서 가중치가 조정되는 속도
batch_size = 32        # 배치 크기: 한 번의 훈련 반복에서 사용되는 샘플의 수
epochs = 100           # 에포크 수: 전체 데이터셋을 훈련하는 반복 횟수
hidden_layer1_size = 128  # 첫 번째 은닉층의 노드 수 결정: 모델의 복잡성 조절
hidden_layer2_size = 64   # 두 번째 은닉층의 노드 수 결정: 모델의 복잡성 조절

# TensorFlow/Keras를 사용하여 신경망 모델 정의
debug_message("신경망 모델 정의 중...")
model = Sequential()
model.add(Dense(hidden_layer1_size, input_dim=X_train.shape[1], activation='relu'))  # ReLU 활성화 함수 사용: sigmoid, tanh, leakyrelu 얘기 해야지
model.add(Dense(hidden_layer2_size, activation='relu'))
model.add(Dense(y_train.shape[1]))
debug_message("신경망 모델 정의 완료")

# 모델 컴파일
debug_message("모델 컴파일 중...")
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='mse')  # Adam 옵티마이저 사용: NAG 사용하는 NAdam, Momentum 사용하는 Adam 중 Adam 사용
debug_message("모델 컴파일 완료")

# 모델 학습
debug_message("모델 학습 중...")
model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)
debug_message("모델 학습 완료")

# 모델 평가
debug_message("모델 평가 중...")
train_loss = model.evaluate(X_train, y_train)
test_loss = model.evaluate(X_test, y_test)
print(f'훈련 손실: {train_loss:.4f}')
print(f'테스트 손실: {test_loss:.4f}')

# 예측 및 평가 (평균 제곱 오차 사용)
debug_message("예측 및 평가 중...")
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)
train_mse = mean_squared_error(y_train, y_pred_train)
test_mse = mean_squared_error(y_test, y_pred_test)
print(f'훈련 MSE: {train_mse:.4f}')
print(f'테스트 MSE: {test_mse:.4f}')
debug_message("예측 및 평가 완료")

# 모델 저장
debug_message("모델 저장 중...")
saved_filename = save_model_with_versioning(model, file_path, "importance_model")
debug_message(f"모델 저장 완료: {saved_filename}")

print(f"Initial importance model created and saved as {saved_filename}.")


##### 작업 긴급도 예측 모델

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import os

# 디버깅 메시지 출력 함수
def debug_message(message):
    print(f"[디버깅] {message}")

# 모델 저장 함수
def save_model_with_versioning(model, file_path, base_filename):
    version = 1
    filename = f"{base_filename}_v{version}.h5"
    while os.path.exists(os.path.join(file_path, filename)):
        version += 1
        filename = f"{base_filename}_v{version}.h5"
    model.save(os.path.join(file_path, filename))
    return filename

# 데이터 로드 및 전처리
file_path = './refer/output/'
debug_message("작업 긴급도 데이터 로드 중...")
data = pd.read_csv(f'{file_path}data.csv')
debug_message("작업 긴급도 데이터 로드 완료")

# 초기 긴급도 기준 생성
debug_message("긴급도 관련 데이터 생성 중...")
data['start_date'] = pd.to_datetime(data['start_date'])
data['end_date'] = pd.to_datetime(data['end_date'])
data['days_left'] = (data['end_date'] - data['start_date']).dt.days
data['urgency'] = 1 / (data['days_left'] + 1)  # 마감일이 가까울수록 긴급도가 높아짐
debug_message("긴급도 관련 데이터 생성 완료")

# 긴급도 모델을 위한 입력 데이터 준비
X_urgency = data[['start_date', 'end_date', 'complexity']].copy()
X_urgency['start_date'] = (X_urgency['start_date'] - X_urgency['start_date'].min()).dt.days
X_urgency['end_date'] = (X_urgency['end_date'] - X_urgency['end_date'].min()).dt.days
y_urgency = data['urgency'].values

# 데이터 정규화
debug_message("데이터 정규화 중...")
scaler = StandardScaler()
X_urgency = scaler.fit_transform(X_urgency)
debug_message("데이터 정규화 완료")

# 데이터를 학습용과 테스트용으로 분할
debug_message("데이터 학습용 및 테스트용 분할 중...")
X_train_urgency, X_test_urgency, y_train_urgency, y_test_urgency = train_test_split(X_urgency, y_urgency, test_size=0.2, random_state=42)
debug_message("데이터 학습용 및 테스트용 분할 완료")

# 하이퍼파라미터 설정
learning_rate = 0.001  # 학습률 결정: 학습 과정에서 가중치가 조정되는 속도
batch_size = 32        # 배치 크기 결정: 한 번의 훈련 반복에서 사용되는 샘플의 수
epochs = 100           # 에포크 수 결정: 전체 데이터셋을 훈련하는 반복 횟수
hidden_layer1_size = 128  # 첫 번째 은닉층의 노드 수 결정: 모델의 복잡성 조절
hidden_layer2_size = 64   # 두 번째 은닉층의 노드 수 결정: 모델의 복잡성 조절

# TensorFlow/Keras를 사용하여 신경망 모델 정의
debug_message("신경망 모델 정의 중...")
urgency_model = Sequential()
urgency_model.add(Dense(hidden_layer1_size, input_dim=X_train_urgency.shape[1], activation='relu'))  # ReLU 활성화 함수 사용: 비선형성을 추가하여 모델의 학습 능력 향상
urgency_model.add(Dense(hidden_layer2_size, activation='relu'))
urgency_model.add(Dense(1))
debug_message("신경망 모델 정의 완료")

# 모델 컴파일
debug_message("모델 컴파일 중...")
urgency_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='mse')  # Adam 옵티마이저 사용: NAdam도 
debug_message("모델 컴파일 완료")

# 모델 학습
debug_message("모델 학습 중...")
urgency_model.fit(X_train_urgency, y_train_urgency, epochs=epochs, batch_size=batch_size, validation_split=0.2)
debug_message("모델 학습 완료")

# 모델 평가
debug_message("모델 평가 중...")
train_loss_urgency = urgency_model.evaluate(X_train_urgency, y_train_urgency)
test_loss_urgency = urgency_model.evaluate(X_test_urgency, y_test_urgency)
print(f'훈련 긴급도 손실: {train_loss_urgency:.4f}')
print(f'테스트 긴급도 손실: {test_loss_urgency:.4f}')

# 예측 및 평가 (평균 제곱 오차 사용)
debug_message("예측 및 평가 중...")
y_pred_train_urgency = urgency_model.predict(X_train_urgency)
y_pred_test_urgency = urgency_model.predict(X_test_urgency)
train_mse_urgency = mean_squared_error(y_train_urgency, y_pred_train_urgency)
test_mse_urgency = mean_squared_error(y_test_urgency, y_pred_test_urgency)
print(f'훈련 긴급도 MSE: {train_mse_urgency:.4f}')
print(f'테스트 긴급도 MSE: {test_mse_urgency:.4f}')
debug_message("예측 및 평가 완료")

# 모델 저장
debug_message("모델 저장 중...")
saved_filename = save_model_with_versioning(urgency_model, file_path, "urgency_model")
debug_message(f"모델 저장 완료: {saved_filename}")

print(f"Initial urgency model created and saved as {saved_filename}.")


##### 시간 분배 및 재분배 계산 모델

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import os
from datetime import datetime

# 디버깅 메시지 출력 함수
def debug_message(message):
    print(f"[디버깅] {message}")

# 모델 로드 함수
def load_latest_model(file_path, base_filename):
    version = 1
    filename = f"{base_filename}_v{version}.h5"
    while os.path.exists(os.path.join(file_path, filename)):
        version += 1
        filename = f"{base_filename}_v{version}.h5"
    if version == 1:
        raise FileNotFoundError(f"No model found for {base_filename}")
    return load_model(os.path.join(file_path, filename))

# 데이터 로드
file_path = './refer/output/'
debug_message("사용자 정보 데이터를 불러오는 중...")
user_info = pd.read_csv(f'{file_path}random_user_info.csv')
debug_message("사용자 정보 데이터 로드 완료")

debug_message("schedule_2 데이터를 불러오는 중...")
schedule_2 = pd.read_csv(f'{file_path}random_schedule_2.csv')
debug_message("schedule_2 데이터 로드 완료")

debug_message("user_check 데이터를 불러오는 중...")
user_check = pd.read_csv(f'{file_path}user_check.csv') # 유저의 작업 달성 정도 or 여부를 받아오기
debug_message("user_check 데이터 로드 완료")

debug_message("user_time 데이터를 불러오는 중...")
user_time = pd.read_csv(f'{file_path}user_time.csv') # 유저의 하루 가용 시간을 받아오기
debug_message("user_time 데이터 로드 완료")

# 오늘 날짜의 daily_time 설정
today_date = datetime.today().strftime('%Y-%m-%d')
daily_time_row = user_time[user_time['date'] == today_date]

if not daily_time_row.empty:
    daily_time = daily_time_row['daily_time'].values[0]
else:
    raise ValueError(f"No daily time data available for today ({today_date})")

# 중요도 모델 및 긴급도 모델 로드
debug_message("중요도 모델 로드 중...")
importance_model = load_latest_model(file_path, "importance_model")
debug_message("중요도 모델 로드 완료")

debug_message("긴급도 모델 로드 중...")
urgency_model = load_latest_model(file_path, "urgency_model")
debug_message("긴급도 모델 로드 완료")

# 범주형 데이터에 대한 원-핫 인코딩
debug_message("범주형 데이터에 대한 원-핫 인코딩 중...")
encoder = OneHotEncoder(handle_unknown='ignore')
encoded_data = encoder.fit_transform(user_info[['gender', 'job', 'mbti']]).toarray()
debug_message("범주형 데이터에 대한 원-핫 인코딩 완료")

# 인코딩된 데이터와 수치형 데이터를 결합
numeric_data = user_info[['age']].values
X_user_info = np.hstack((encoded_data, numeric_data))

# 데이터 정규화
debug_message("데이터 정규화 중...")
scaler = StandardScaler()
X_user_info = scaler.fit_transform(X_user_info)
debug_message("데이터 정규화 완료")

# 중요도 예측
debug_message("중요도 예측 중...")
importance_scores = importance_model.predict(X_user_info)
debug_message("중요도 예측 완료")

# 긴급도 예측
debug_message("긴급도 예측 중...")
X_schedule = schedule_2[['start_date', 'end_date', 'complexity']].copy()
X_schedule['start_date'] = pd.to_datetime(X_schedule['start_date'])
X_schedule['end_date'] = pd.to_datetime(X_schedule['end_date'])
X_schedule['start_date'] = (X_schedule['start_date'] - X_schedule['start_date'].min()).dt.days
X_schedule['end_date'] = (X_schedule['end_date'] - X_schedule['end_date'].min()).dt.days
X_schedule = scaler.fit_transform(X_schedule)
urgency_scores = urgency_model.predict(X_schedule)
debug_message("긴급도 예측 완료")

# 중요도와 긴급도를 기반으로 작업 분배
debug_message("작업 분배 중...")
total_importance = np.sum(importance_scores)
total_urgency = np.sum(urgency_scores)
weights = (importance_scores / total_importance) + (urgency_scores / total_urgency)
weights /= np.sum(weights)  # 가중치의 합이 1이 되도록 정규화

# 작업 시간 분배
work_distribution = daily_time * weights.flatten()
debug_message(f"작업 시간 분배 완료: {work_distribution}")

# 유저의 작업 달성 정도를 반영한 재분배
debug_message("유저의 작업 달성 정도 반영한 재분배 중...")
achievement_ratios = user_check['achievement_ratio'].values
adjusted_distribution = work_distribution * achievement_ratios
adjusted_distribution /= np.sum(adjusted_distribution)  # 조정된 분배 시간의 합이 daily_time이 되도록 정규화
adjusted_work_distribution = daily_time * adjusted_distribution
debug_message(f"조정된 작업 시간 분배 완료: {adjusted_work_distribution}")

# 결과 출력
print(f"초기 작업 시간 분배: {work_distribution}")
print(f"조정된 작업 시간 분배: {adjusted_work_distribution}")
